In [1]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import cv2
import dlib
import time
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Load model

In [2]:
detector = dlib.get_frontal_face_detector()
p = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor (p)
face_rec_model_path = 'dlib_face_recognition_resnet_model_v1.dat'
facerec = dlib.face_recognition_model_v1(face_rec_model_path)


## get facial imformation 

If an image contains no face, it will not be included in the pkl file

In [3]:
result = []
def get_faces(file):
    image = cv2.imdecode(np.fromfile(file,dtype=np.uint8),-1)
    image = cv2. cvtColor (image, cv2.COLOR_BGR2RGB)
    gray = cv2. cvtColor (image, cv2.COLOR_RGB2GRAY)

    rects = detector (gray , 0)
    if len(rects) >=0:
        for rect in rects :
            shape = predictor (gray , rect )
            face_chip = dlib.get_face_chip(image, shape)
            face_descriptor = facerec.compute_face_descriptor(face_chip)
            item = [file, [rect.left(), rect.top(), rect.width(), rect.height()], list(face_descriptor)]
            result.append(item)
        
def recu_dir(org_dir):
    for file in os.listdir(org_dir):
        if os.path.isdir(os.path.join(org_dir,file)):
            abspath = os.path.join(org_dir,file)
            recu_dir(os.path.join(org_dir, file))
        else:
            if file[-4:] == '.JPG':
                get_faces(os.path.join(org_dir, file))


In [4]:
recu_dir('D://Data2')
data = pd.DataFrame(result, columns=['file', 'rect', 'face_descriptor'])
data.head()

,file,rect,face_descriptor
0,D://Data2\new\2019.08.31机器之心\上午\BB4A4589.JPG,"[369, 189, 126, 126]","[-0.08341855555772781, 0.0792071670293808, 0.0..."
1,D://Data2\new\2019.08.31机器之心\上午\BB4A4590.JPG,"[532, 234, 180, 180]","[-0.05788932740688324, 0.05817071348428726, 0...."
2,D://Data2\new\2019.08.31机器之心\上午\BB4A4591.JPG,"[453, 234, 180, 180]","[-0.10730400681495667, 0.08268777281045914, 0...."
3,D://Data2\new\2019.08.31机器之心\上午\BB4A4594.JPG,"[229, 301, 73, 73]","[-0.12571413815021515, 0.09957826137542725, 0...."
4,D://Data2\new\2019.08.31机器之心\上午\BB4A4594.JPG,"[765, 246, 87, 88]","[-0.12546943128108978, 0.0979333147406578, 0.1..."


In [5]:
data.to_pickle('faces_all.pkl')